# Visualização da Informação
## Escola de Matemática Aplicada - Fundação Getúlio Vargas
## Mestrado em Modelagem Matemática

Aluno: Gianlucca Devigili<br>
Github do projeto: https://github.com/GDevigili/information-visualization-homeworks

## Trabalho 2: Análise e reprodução de uma visualização reconhecida ou relevante historicamente

+ Parte1: Encontrar os dados (compartilhar referências de dados no slack)
+ Parte2: Fazer uma análise de qual seria a função pretendida com a visualização proposta. 
+ Parte3: Fazer uma reprodução da visualização escolhida utilizando uma ferramenta computacional atual (de preferência a mesma escolhida por vocês no trabalho 1)
+ Parte4: Propor alguma modificação (fundamentando conceitualmente) na visualização proposta. Exemplo: Incluir anotação, incluir interatividade, modificar título ou legenda, adicionar informação, etc.

In [1]:
import pandas as pd
import altair as alt
import plotly.express as px
from plotly.subplots import make_subplots

import vega_datasets

from altair_saver import save

A visualização escolhida foi a de **Florence Nightingale** referente às __causas de morte na guerra da Crimeia__ (1853-1856)


![](https://wp-assets.highcharts.com/www-highcharts-com/blog/wp-content/uploads/2015/05/30154347/Nightingale%E2%80%99s-diagram-1.jpg)

## (1) Aquisição dos dados

Para importar os dados, utilizei a biblioteca `vega_datasets`. <br>Inicialmente eu havia realizado um web-scrapping dos dados ([como pode ser visto neste commit]([https://github.com/GDevigili/information-visualization-homeworks/commit/a2c97beb6c5c5f1e174c854f094720b09437893a#diff-f485367de3abdc640726199ca3360f1fdfc1043bf3abdde1ecf95faa3556adf2)), porém ao plotar o gráfico percebi que os dados da fonte que eu peguei estavam errados e não reproduziam o gráfico de Nightingale, então preferi usar a biblioteca.

In [34]:
# Carrega os dados
df_crimea = vega_datasets.data.crimea()
df_crimea

,date,wounds,other,disease
0,1854-04-01,0,110,110
1,1854-05-01,0,95,105
2,1854-06-01,0,40,95
3,1854-07-01,0,140,520
4,1854-08-01,20,150,800
5,1854-09-01,220,230,740
6,1854-10-01,305,310,600
7,1854-11-01,480,290,820
8,1854-12-01,295,310,1100
9,1855-01-01,230,460,1440


### Preparação dos dados

Para reproduzir o gráfico, precisamos dividi os dados em dois intervalos de tempo, sendo o primeiro indo de abril de 1854 até março de 1855 e o segundo de abril de 1855 até março de 1856.<br>
Após isso, usei o método `pd.melt` para transformar o dataset de modo que ele tenha 3 colunas: `Date`, `Death` e `Cause`. Os dados então ficam da maneira apresentada abaixo:

In [35]:
causes = ['other', 'wounds', 'disease']

# Transforma o dataset completo em um dataset melted
df_melted = pd.melt(
        df_crimea, 
        id_vars = ['date'], 
        value_vars = causes,
        var_name = 'cause', value_name = 'deaths'
        )


# Formata a data no formato 'Apr 1854'
df_crimea['date'] = [date.strftime('%b %Y') for date in df_crimea['date']]

# Cria um dataset para cada período de 12 mesmos
df1 = pd.melt(
        df_crimea[:12], 
        id_vars = ['date'], 
        value_vars = causes,
        var_name = 'cause', value_name = 'deaths'
        )
df2 = pd.melt(
        df_crimea[12:], 
        id_vars = ['date'], 
        value_vars = causes,
        var_name = 'cause', value_name = 'deaths'
        )

# Remove o dia do dataset melted
df_melted['date'] = [date.strftime('%Y-%m') for date in df_melted['date']]

# Apresenta o dataset
df_melted.head()

,date,cause,deaths
0,1854-04,other,110
1,1854-05,other,95
2,1854-06,other,40
3,1854-07,other,140
4,1854-08,other,150


## (2) Análise da Função da Visualização


## (3) Reprodução da Visualização

In [15]:
colors = ['#fac0bd', '#4c3933', '#b0c3d1']

fig1 = px.bar_polar(
    df1, 
    r = 'deaths',
    color = 'cause', 
    theta = 'date',
    start_angle = 165,
    color_discrete_sequence = colors,
    template = 'simple_white'
)

fig1.show()

fig2 = px.bar_polar(
    df2, 
    r = 'deaths',
    color = 'cause', 
    theta = 'date',
    start_angle = 165,
    color_discrete_sequence = colors,
    template = 'simple_white'
)

fig2.show()

## (4) Modificações Propostas

O tipo de visualização escolhida por Nightingale, apesar de muito agradável visualmente, não consegue representar com precisão os dados, por exemplo, se invertermos a ordem dos dados o gráfico, parece que trocamos os valores numéricos dele:

In [16]:
causes.reverse()
colors.reverse()

df_aux = pd.melt(
        df_crimea[:12], 
        id_vars = ['date'], 
        value_vars = causes,
        var_name = 'cause', value_name = 'deaths'
        )

fig1 = px.bar_polar(
    df_aux, 
    r = 'deaths',
    color = 'cause', 
    theta = 'date',
    start_angle = 165,
    color_discrete_sequence = colors,
    template = 'simple_white', width = 800, height = 400
)

fig1.show()

Além disso, os meses que tem valores pequenos, como abril, maio e junho de 1854, são quase que invisíveis no gráfico.

Uma modificação possível seria trocar o gráfico de barras em coordenadas polares por um gráfico de barras comum:

In [42]:
bar1 = alt.Chart(df_melted).mark_bar().encode(
    x = 'date',
    y = 'deaths',
    color = 'cause'
).properties(width = 800, height = 400, title="Gráfico de Barras das Causas de Morte no exército do leste").interactive()

save(bar1, "bar1.html")

bar1

alt.Chart(...)

Com este gráfico a análise fica mais fácil, tornando possível perceber que alguns meses não tem mortes por ferimentos (o que em alguns casos não ficava claro na versão radial) e também é possível observar a redução no número de mortes ao longo do tempo. Além disso optei por unir os dois gráficos de modo a apresentar os dados como um todo, já que a separação por ano não faz tanto sentido agora que a compactação dos dados não é um problema como no bar plot radial.

Outra opção seria, ao invés do gráfico stacked, seria separar as barras por categoria:

In [ ]:
bar2 = alt.Chart(df_melted).mark_bar().encode(
    x = 'date',
    y = 'deaths'
    color = 'cause'
)